In [1]:
# Code is from 
# https://github.com/milvus-io/bootcamp/blob/master/integration/build_RAG_with_milvus_and_docling.ipynb
# Joel J Varghese

import os
api = os.getenv("HF_TOKEN")

In [3]:
from transformers import pipeline, AutoTokenizer

base_model = "meta-llama/Llama-3.2-1B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(base_model)
pipe = pipeline("text-generation", model=base_model, tokenizer=tokenizer, device_map="auto")

Some parameters are on the meta device because they were offloaded to the disk.
Device set to use mps


In [4]:
from sentence_transformers import SentenceTransformer
import numpy as np
import torch

model_name = "BAAI/bge-large-en-v1.5"
DEVICE = torch.device('cuda:3' if torch.cuda.is_available() else 'cpu')
encoder = SentenceTransformer(model_name, device=DEVICE)

def emb_text(text):
    embeddings = encoder.encode(text, convert_to_numpy=True, normalize_embeddings=True)
    return embeddings

In [5]:
test_embedding = emb_text("This is a test")
embedding_dim = len(test_embedding)
print(embedding_dim)
print(test_embedding[:10])

1024
[ 0.01152764  0.02975923  0.00379159  0.03594419 -0.01584916 -0.01495779
 -0.01805204 -0.00275517  0.03082995  0.03400182]


In [6]:
from docling.document_converter import DocumentConverter
from docling_core.transforms.chunker import HierarchicalChunker

converter = DocumentConverter()
chunker = HierarchicalChunker()

source = "https://milvus.io/docs/overview.md"
doc = converter.convert(source).document

texts = [chunk.text for chunk in chunker.chunk(doc)]

for i, text in enumerate(texts[:5]):
    print(f"Chunk {i+1}:\n{text}\n{'-'*50}")

2025-10-24 15:02:16,065 - INFO - detected formats: [<InputFormat.MD: 'md'>]
2025-10-24 15:02:16,077 - INFO - Going to convert document batch...
2025-10-24 15:02:16,078 - INFO - Initializing pipeline for SimplePipeline with options hash 995a146ad601044538e6a923bea22f4e
2025-10-24 15:02:16,089 - INFO - Loading plugin 'docling_defaults'
2025-10-24 15:02:16,091 - INFO - Registered picture descriptions: ['vlm', 'api']
2025-10-24 15:02:16,092 - INFO - Processing document overview.md
2025-10-24 15:02:16,491 - INFO - Finished converting document overview.md in 0.78 sec.


Chunk 1:
Milvus is a bird of prey in the genus Milvus of the hawk family Accipaitridae, celebrated for its speed in flight, keen vision, and remarkable adaptability.
--------------------------------------------------
Chunk 2:
Zilliz adopts the name Milvus for its open-source high-performance, highly scalable vector database that runs efficiently across a wide range of environments, from a laptop to large-scale distributed systems. It is available as both open-source software and a cloud service.
--------------------------------------------------
Chunk 3:
Developed by Zilliz and soon donated to the LF AI & Data Foundation under the Linux Foundation, Milvus has become one of the world's leading open-source vector database projects. It is distributed under the Apache 2.0 license, and most contributors are experts from the high-performance computing (HPC) community, specializing in building large-scale systems and optimizing hardware-aware code. Core contributors include professionals from

In [7]:
import pymilvus
from pymilvus import connections

print(f"pymilvus: {pymilvus.__version__}")
connections.connect("default", host="localhost", port="19530")

from pymilvus import MilvusClient
mc = MilvusClient("milvus_demo.db")

collection_name = "my_rag_collection"

pymilvus: 2.5.16


/Users/joeljvarghese/Documents/Workspace/Milvus_ollama_trial/.venv/lib/python3.12/site-packages/milvus_lite/__init__.py:15: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution
I0000 00:00:1761332542.729863  955274 fork_posix.cc:71] Other threads are currently calling into gRPC, skipping fork() handlers
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [8]:
if mc.has_collection(collection_name):
    mc.drop_collection(collection_name)

In [9]:
mc.create_collection(
    collection_name = collection_name,
    dimension=embedding_dim,
    metric_type="IP"
)

In [10]:
from tqdm import tqdm

data = []
for i, chunk in enumerate(tqdm(texts, desc="Processing chunks")):
    embedding = emb_text(chunk)
    data.append({"id": i, "vector": embedding, "text": chunk})

mc.insert(collection_name=collection_name, data=data)

Processing chunks:   0%|          | 0/41 [00:00<?, ?it/s]

Processing chunks: 100%|██████████| 41/41 [00:16<00:00,  2.53it/s]


{'insert_count': 41, 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40], 'cost': 0}

In [11]:
question = (
    "What are the three deployment modes of Milvus, and what are their differences"
)

In [12]:
search_res = mc.search(
    collection_name=collection_name,
    data=[emb_text(question)],
    limit=3,
    search_params={"metric_type": "IP", "params": {}},
    output_fields=["text"],
)

Batches: 100%|██████████| 1/1 [00:02<00:00,  2.76s/it]


In [13]:
import json

retrieved_lines_with_distances = [
    (res["entity"]["text"], res["distance"]) for res in search_res[0]
]
print(json.dumps(retrieved_lines_with_distances, indent=4))

[
    [
        "Milvus offers three deployment modes, covering a wide range of data scales-from local prototyping in Jupyter Notebooks to massive Kubernetes clusters managing tens of billions of vectors:",
        0.7934890389442444
    ],
    [
        "- Milvus Lite is a Python library that can be easily integrated into your applications. As a lightweight version of Milvus, it's ideal for quick prototyping in Jupyter Notebooks or running on edge devices with limited resources. [Learn more](/docs/milvus_lite.md) .\n- Milvus Standalone is a single-machine server deployment, with all components bundled into a single Docker image for convenient deployment. [Learn more](/docs/install_standalone-docker.md) .\n- Milvus Distributed can be deployed on Kubernetes clusters, featuring a cloud-native architecture designed for billion-scale or even larger scenarios. This architecture ensures redundancy in critical components. [Learn more](/docs/install_cluster-milvusoperator.md) .",
        0.753

In [14]:
context = "\n".join(
    [line_with_distance[0] for line_with_distance in retrieved_lines_with_distances] 
)

In [15]:
SYSTEM_PROMPT = """
Human: You are an AI assistant. You are able to find answers to the questions from the contextual passage snippet provided.
"""
USER_PROMPT = f"""
Use the following pieces of information enclosed in <context> tags to provide an answer to the question enclosed in <question> tags.
<context>
{context}
</context>
<question>
{question}
</question>
"""

In [16]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(base_model, device_map="auto")

prompt = f"{SYSTEM_PROMPT}\nUser: {USER_PROMPT}\nAssistant:"

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=300)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

assistant_reply = response.split("Assistant:")[-1].strip()
print(assistant_reply)

2025-10-24 15:17:09,779 - WARNING - Some parameters are on the meta device because they were offloaded to the disk.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


KeyboardInterrupt: 